In [1]:
import pandas as pd
import numpy as np

In [2]:
set2 = pd.read_csv('lv2-2401.csv')
set2

,C_ID,GENDER,AGE,INCOME,JOB,WORK_EXP,FAMILY,SPENDING,GRADE
0,1,Male,19,15000,B,1,4,11.7,prestige
1,2,Male,21,35000,D,3,3,24.3,royal blue
2,3,Female,20,86000,D,1,1,1.8,general
3,4,Female,23,59000,G,0,2,23.1,royal blue
4,5,Female,31,38000,C,2,6,12.0,prestige
...,...,...,...,...,...,...,...,...,...
1995,1996,Female,41,184387,A,8,7,12.0,prestige
1996,1997,Female,61,73158,E,7,7,9.6,star
1997,1998,Male,57,90961,B,9,2,4.2,star
1998,1999,Male,47,182109,F,7,2,1.2,general


In [4]:
set2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
C_ID        2000 non-null int64
GENDER      2000 non-null object
AGE         2000 non-null int64
INCOME      2000 non-null int64
JOB         1965 non-null object
WORK_EXP    2000 non-null int64
FAMILY      2000 non-null int64
SPENDING    2000 non-null float64
GRADE       2000 non-null object
dtypes: float64(1), int64(5), object(3)
memory usage: 140.8+ KB


### Q1

In [5]:
df4 = set2.copy()

# 1-1.
df4['AGE_P']=df4['AGE'].apply(lambda x : '10s' if x<20 else ('20s' if (x>=20)&(x<30) else ('30s' if (x>=30)&(x<40)\
                                                                                              else ('40s' if (x>=40)&(x<50)\
                                                                                                   else ('other' if x>=60 else '50s')))))

In [6]:
agg240 = df4.groupby(['AGE_P'])['SPENDING'].mean().reset_index()
agg241 = df4.groupby(['AGE_P'])['C_ID'].count().reset_index()
Agg24 = agg240.merge(agg241, on='AGE_P')
Agg24.columns = ['AGE_P','AVG_SPEND','CUST_CNT']
Agg24

,AGE_P,AVG_SPEND,CUST_CNT
0,10s,16.125789,380
1,20s,15.665403,211
2,30s,15.134440,241
3,40s,14.432320,362
4,50s,15.314179,402
5,other,15.138861,404


In [7]:
# 1-5
ans24 = Agg24.iloc[Agg24.AVG_SPEND.idxmax()].CUST_CNT

print('Q1 Answer:',ans24)

Q1 Answer: 380


### Q2

In [8]:
df5 = set2.copy()
df5=df5.sort_values(by='C_ID')

# 2-1.
var5 = ['AGE', 'INCOME', 'WORK_EXP', 'FAMILY', 'SPENDING']

from sklearn.preprocessing import StandardScaler

df5s = pd.DataFrame(StandardScaler().fit(df5[var5]).transform(df5[var5]), columns=var5)
df5s['GRADE']=df5['GRADE']



In [9]:
# 2-2, 2-3
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

res5 = []
for i in range(3,5):
    pred = KMeans(random_state=1234, init='random', n_clusters=i).fit(df5s[var5]).predict(df5s[var5])
    ss = silhouette_score(df5s[var5], pred)
    print(i,':', ss)
    res5.append([i,ss])

res5_df = pd.DataFrame(res5, columns=['i','ss']).sort_values('ss', ascending=False)
max_sil = res5_df.iloc[0].ss
ans25 = round(max_sil, 3)

print('Q2 Result:',max_sil)
print('Q2 Answer:', ans25)

3 : 0.15775962870715157
4 : 0.15510749408111774
Q2 Result: 0.15775962870715157
Q2 Answer: 0.158


### Q3

In [10]:
# 3-1.
df6=set2.copy()

df6f = df6[df6['AGE']>=14]
df6f = df6f[~df6f['JOB'].isnull()]
print(df6f.shape)

(1714, 9)


In [11]:
# 3-2.
df6d = pd.concat([df6f, pd.get_dummies(df6f[['GENDER', 'JOB']], drop_first=True)], axis=1).drop(['GENDER','JOB'], axis=1)
print(df6d.shape)

(1714, 16)


In [12]:
# 3-3.
train6 = df6d[df6d['C_ID']%3!=0]
test6 = df6d[df6d['C_ID']%3==0]
print(train6.shape)
print(test6.shape)

xtrain6 = train6.drop(['C_ID','GRADE', 'SPENDING'], axis=1)
ytrain6 = train6['GRADE']
xtest6 = test6.drop(['C_ID','GRADE', 'SPENDING'], axis=1)
ytest6 = test6['GRADE']

(1148, 16)
(566, 16)


In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 3-4.
dt6 = DecisionTreeClassifier(random_state=1234, criterion='gini', max_depth=5).fit(xtrain6, ytrain6)

# 3-5.
pred6 = dt6.predict(xtest6)
a6 = accuracy_score(ytest6, pred6)*100
ans26 = round(a6, 2)

print('Q3 Result:',a6)
print('Q3 Answer:', ans26)

Q3 Result: 33.215547703180206
Q3 Answer: 33.22


### Answer Summary

In [15]:
print(ans24)
print(ans25)
print(ans26)

380
0.158
33.22
